# CRNN model based on spectrogram approach for AF classification
The following notebook presents a CRNN model used for AF classifications.

## Importing libraries

In [30]:
import numpy as np
import keras
import pickle
import sys
import random
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Dropout, Flatten, Lambda, Reshape, Bidirectional, LSTM, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc, accuracy_score, precision_recall_curve
import matplotlib.pyplot as plt

## Loading datasets

In [11]:
def LoadTrainingSet(filename):

    with open(filename, 'rb') as f:
        xTrain, yTrain = pickle.load(f)
        
        yTrain[yTrain=='N'] = 0
        yTrain[yTrain=='A'] = 1
        yTrain[yTrain=='O'] = 2
        yTrain[yTrain=='~'] = 3

        # Count the elements in the sets
        num_train_data_normal = sum(yTrain == 0)
        num_train_data_afib   = sum(yTrain == 1)
        num_train_data_other = sum(yTrain == 2)
        num_train_data_noise   = sum(yTrain == 3)

        print('### TRAIN SET')
        print('\tNormal ECG: {} ({:.2f}%)'.format(num_train_data_normal, 100 * num_train_data_normal / len(yTrain)))
        print('\tAfib ECG: {} ({:.2f}%)'.format(num_train_data_afib, 100 * num_train_data_afib / len(yTrain)))
        print('\tOther ECG: {} ({:.2f}%)'.format(num_train_data_other, 100 * num_train_data_other / len(yTrain)))
        print('\tNoisy ECG: {} ({:.2f}%)'.format(num_train_data_noise, 100 * num_train_data_noise / len(yTrain)))
        

        yTrain = keras.utils.to_categorical(yTrain)

        return xTrain, yTrain

def LoadTestSet(filename):

    with open(filename, 'rb') as f:
        xTest, yTest = pickle.load(f)

        yTest[yTest=='N'] = 0
        yTest[yTest=='A'] = 1
        yTest[yTest=='O'] = 2
        yTest[yTest=='~'] = 3

        num_val_data_normal   = sum(yTest == 0)
        num_val_data_afib     = sum(yTest == 1)
        num_val_data_other = sum(yTest == 2)
        num_val_data_noise   = sum(yTest == 3)

        print('### VALIDATION SET')
        print('\tNormal ECG: {} ({:.2f}%)'.format(num_val_data_normal, 100 * num_val_data_normal / len(yTest)))
        print('\tAfib ECG: {} ({:.2f}%)'.format(num_val_data_afib, 100 * num_val_data_afib / len(yTest)))
        print('\tOther ECG: {} ({:.2f}%)'.format(num_val_data_other, 100 * num_val_data_other / len(yTest)))
        print('\tNoisy ECG: {} ({:.2f}%)'.format(num_val_data_noise, 100 * num_val_data_noise / len(yTest)))

        yTest = keras.utils.to_categorical(yTest)

        return xTest, yTest

## Defining Generator methods
Since dataset takes some space, we decided to use a generator which produces batches of 32 observations each

In [12]:
def AugGenerator(xTrain, xTest, yTrain, yTest):

    imagegen = ImageDataGenerator()

    trainGenerator = imagegen.flow(xTrain, yTrain, batch_size=20)
    testGenerator = imagegen.flow(xTest, yTest, batch_size=20)

    return trainGenerator, testGenerator

## Model creation
We are now going to create the CRNN

In [13]:
def CRNN(blockSize, blockCount, inputShape):

    model = Sequential()

    channels = 32
    for i in range(blockCount):
        for j in range(blockSize):
            if i == 0 and j == 0:
                conv = Conv2D(channels, kernel_size=(5, 5),
                              input_shape=inputShape, padding='same')
            else:
                conv = Conv2D(channels, kernel_size=(5, 5), padding='same')
            model.add(conv)
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            model.add(Dropout(0.3))
            if j == blockSize - 2:
                channels += 32
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(Dropout(0.3))

    model.add(Reshape((3, 224)))

    # LSTM layer
    model.add(LSTM(200))
    model.add(Dropout(0.5))

    # Adding noise
    model.add(GaussianNoise(0.2))

    # Linear classifier
    model.add(Dense(4, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy']) 

    print(model.summary())
    return model

## Model training function

In [21]:
def TrainCRNN(model, epochs):

    xTrain, yTrain = LoadTrainingSet('./TrainingSetFFT.pk1')
    xTest, yTest = LoadTestSet('./TestSetFFT.pk1')
    
    trainGen, testGen = AugGenerator(xTrain, xTest, yTrain, yTest)

    # Checkpoint
    filepath="./weights-crnn-{epoch:02d}-{val_acc:.2f}.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    model.fit_generator(trainGen,
                        validation_data=testGen, steps_per_epoch = int(np.ceil(4040 / 20)),
                        validation_steps = int(np.ceil(1010/20)),
                        epochs=epochs, callbacks=callbacks_list, verbose=1)

    model.save('./crnn_model.h5')

## Model creation

In [22]:
model = CRNN(4, 6, (140, 33, 1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 140, 33, 32)       832       
_________________________________________________________________
batch_normalization_49 (Batc (None, 140, 33, 32)       128       
_________________________________________________________________
activation_49 (Activation)   (None, 140, 33, 32)       0         
_________________________________________________________________
dropout_63 (Dropout)         (None, 140, 33, 32)       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 140, 33, 32)       25632     
_________________________________________________________________
batch_normalization_50 (Batc (None, 140, 33, 32)       128       
_________________________________________________________________
activation_50 (Activation)   (None, 140, 33, 32)       0         
__________

## Starting the training

In [23]:
TrainCRNN(model, 1)

### TRAIN SET
	Normal ECG: 4040 (52.89%)
	Afib ECG: 1180 (15.45%)
	Other ECG: 1965 (25.72%)
	Noisy ECG: 454 (5.94%)
### VALIDATION SET
	Normal ECG: 1010 (59.20%)
	Afib ECG: 148 (8.68%)
	Other ECG: 491 (28.78%)
	Noisy ECG: 57 (3.34%)
Epoch 1/1
202/202 [==============================] - 528s 3s/step - loss: 1.1405 - acc: 0.5225 - val_loss: 1.0033 - val_acc: 0.5922

Epoch 00001: val_acc improved from -inf to 0.59216, saving model to ./weights-crnn-01-0.59.h5


## Predicting some signals

In [49]:
def EvaluateCRNN(model, weightsFile):

    xTest, yTest = LoadTestSet('./TestSetFFT.pk1')
    
    model.load_weights(weightsFile)

    testIndex = random.randint(0,len(xTest))
    testSignal = xTest[testIndex]
    testSignal = np.expand_dims(testSignal, axis = 0)
    testClass = np.array(yTest[testIndex])
    # testClass = testClass.argmax(axis = 1)
    
    print('Predicting observation number ', str(testIndex), ' which belongs to class: ', str(testClass))
    
    print('Evaluation...')
    yPredictedProbs = model.predict(testSignal)
    yMaxPredictedProbs = np.amax(yPredictedProbs, axis=1)
    yPredicted = yPredictedProbs.argmax(axis = 1)
    
    print('Predicted class is: ', str(yPredicted))

In [50]:
EvaluateCRNN(model, './weights-crnn-01-0.59.h5')

### VALIDATION SET
	Normal ECG: 1010 (59.20%)
	Afib ECG: 148 (8.68%)
	Other ECG: 491 (28.78%)
	Noisy ECG: 57 (3.34%)
Predicting observation number  478  which belongs to class:  [1. 0. 0. 0.]
Evaluation...
Predicted class is:  [0]
